# Priprema

Učitavanje neophodnih biblioteka i inicijalizacija tokenizatora i jezičkog modela

In [1]:
import sys
main_folder = '/'.join(sys.path[0].split('/')[:-1])
sys.path.append(main_folder)

In [2]:
from simpletransformers.ner import ner_model
import json
import classla
from helpers.lemmatizer import load_dictionary, get_lemma, DIALECT_INDEX_EKAVICA, DIALECT_INDEX_IJEKAVICA

with open('labels.json', 'r') as label_file:
    labels = json.load(label_file)
    
model = ner_model.NERModel('electra',  'ICEF-NLP/bcms-bertic-comtext-sr-legal-msd-ekavica', 
                           use_cuda=False, ignore_mismatched_sizes=True, labels=labels)

classla.download('sr', processors='tokenize')
nlp = classla.Pipeline('sr', processors='tokenize')

2024-03-18 13:10:02 INFO: Downloading these customized packages for language: sr (Serbian)...
| Processor | Package  |
------------------------
| tokenize  | standard |

2024-03-18 13:10:02 INFO: Finished downloading models and saved to /Users/lenka/classla_resources.
2024-03-18 13:10:02 INFO: Loading these models for language: sr (Serbian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2024-03-18 13:10:02 INFO: Use device: cpu
2024-03-18 13:10:02 INFO: Loading: tokenize
2024-03-18 13:10:02 INFO: Done loading processors!


# Tokenizacija rečenica

Tokenizacija se vrši uz pomoć CLASSLA tokenizatora za srpski jezik

In [3]:
examples = ['Sudija je izdao propis 1. klase i 2. reda.', 
            'Sud je po prijemu tužbe započeo pripreme za glavnu raspravu koje su obuhvatale i dostavu tužbe tuženom na odgovor u skladu odredbi člana 274 Zakona o parničnom postupku.',
            'OBAVEZUJE SE tuženi Zorić Andrija iz Beograda ulica Sarajevska br. 101 da tužilji Kostić Mari iz Beograda ul. Miše Vujića br. 2 vrati iznos od 145.000 evra u roku od 15 dana od dana prijema pisanog otpravka presude pod pretnjom izvršenja.']


classla_tokenized = []

for sentence in examples:

    doc = nlp(sentence)
    tokenized_sentence = []
    for word in doc.iter_words():
        tokenized_sentence.append(word.text)
    classla_tokenized.append(tokenized_sentence)


print(classla_tokenized)

[['Sudija', 'je', 'izdao', 'propis', '1.', 'klase', 'i', '2.', 'reda', '.'], ['Sud', 'je', 'po', 'prijemu', 'tužbe', 'započeo', 'pripreme', 'za', 'glavnu', 'raspravu', 'koje', 'su', 'obuhvatale', 'i', 'dostavu', 'tužbe', 'tuženom', 'na', 'odgovor', 'u', 'skladu', 'odredbi', 'člana', '274', 'Zakona', 'o', 'parničnom', 'postupku', '.'], ['OBAVEZUJE', 'SE', 'tuženi', 'Zorić', 'Andrija', 'iz', 'Beograda', 'ulica', 'Sarajevska', 'br.', '101', 'da', 'tužilji', 'Kostić', 'Mari', 'iz', 'Beograda', 'ul', '.', 'Miše', 'Vujića', 'br.', '2', 'vrati', 'iznos', 'od', '145.000', 'evra', 'u', 'roku', 'od', '15', 'dana', 'od', 'dana', 'prijema', 'pisanog', 'otpravka', 'presude', 'pod', 'pretnjom', 'izvršenja', '.']]


# Učitavanje rečnika za lematizaciju

Kako se ne bi učitavao ceo rečnik u memoriju, učitavaju se samo podaci vezani za reči koje se nalaze u primerima

In [4]:
DICTIONARY_PATHS = ['srLex_v1.3', 'hrLex_v1.3']
wordlist = [j for i in classla_tokenized for j in i]
load_dictionary(DICTIONARY_PATHS[DIALECT_INDEX_EKAVICA], wordlist, DIALECT_INDEX_EKAVICA, 'ICEF-NLP/bcms-bertic-comtext-sr-legal-msd-ekavica')

FileNotFoundError: [Errno 2] No such file or directory: 'srLex_v1.3'

# Korišćenje modela za morfološko tagovanje i lematizaciju

Jezički model za svaki token predvidja morfološku oznaku, uz pomoć koje se vrši pretraga u rečniku i tako se dobija i lema za svaku reč

In [ ]:
preds_list, model_outputs = model.predict(classla_tokenized, split_on_space=False)

for word in preds_list[0]:
    token = list(word.keys())[0]
    tag = list(word.values())[0]
    lemma = get_lemma(list(word.values())[0], list(word.keys())[0], DIALECT_INDEX_EKAVICA)['text']
    print(token, tag, lemma)
    